<변경점>  
1. Mobilefacenet 사용함. 0.1sec정도 소요됨

1. 사람 + 총 + 칼 인식    
2. Arcface를 사람마다 첫 20프레임 동안 적용(Me, NotME).   
3. NotMe이면서 사람과 총 혹은 칼의 bounding box가 겹치는 경우에는 위험인으로 분류 -> 한 번 위험인으로 분류된 사람의 ID는 dangerous_ids로 관리됨 -> dangerous_ids 중 하나에 해당하는 사람은 위에 "Dangerous person"이라고 뜸
4. pose estimation을 dangerous person에 대해 적용(왼팔들기, 오른팔들기, 양팔들기)  
5. 출력화면 사이즈를 키움  
6. 버그수정: 초기에 무기와 교차하여 dangerous_ids에 들어갔더라도, ArcFace로 내 얼굴임이 확인되면(=나 자신이 무기를 소지한 상황) 그 사람을 위험 인물에서 제외  
7. Kobukki robot이 dangerous_ids 내에 있는 사람을 쫓아가도록 하면 될 듯
8. warning 해결: pip install --upgrade albumentations
9. 사람인 경우에 segmentation으로 윤곽선 치기(성공)


In [2]:
import cv2
import numpy as np
import os
import insightface
from insightface.app import FaceAnalysis

# 1) MobileFaceNet 모델(InsightFace) 초기화
app = FaceAnalysis(name="buffalo_s")  # buffalo_s = MobileFaceNet 계열
app.prepare(ctx_id=0, det_size=(640,640))  # GPU 사용(ctx_id=0), CPU는 -1

def get_mobileface_embedding(image_bgr):
    # image_bgr: np.array(BGR)
    faces = app.get(image_bgr)
    if len(faces)==0:
        return None
    # 보통 faces[0].embedding shape = (128,) or (512,) 모델에 따라 다름
    return faces[0].embedding

# 내 얼굴 사진 폴더
my_face_folder = "C:/Users/idea0/EE101/Jongsul/myface"
embeddings = []

for file in os.listdir(my_face_folder):
    if file.lower().endswith(('.jpg','.png','.jpeg')):
        path = os.path.join(my_face_folder, file)
        img_bgr = cv2.imread(path)
        if img_bgr is None:
            continue
        emb = get_mobileface_embedding(img_bgr)
        if emb is not None:
            embeddings.append(emb)

if len(embeddings)==0:
    raise ValueError("No embeddings generated from MobileFaceNet.")

# 평균 임베딩
avg_embedding = np.mean(embeddings, axis=0)
print("MobileFaceNet embedding shape:", avg_embedding.shape)

# 파일 저장
np.save("my_mobileface_embedding.npy", avg_embedding)
print("Saved my_mobileface_embedding.npy")

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127

C:\Users\idea0\AppData\Roaming\Python\Python38\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [6]:
import cv2
import mediapipe as mp
import time
import torch
import numpy as np
import warnings

from ultralytics import YOLO
from sklearn.metrics.pairwise import cosine_similarity
from deep_sort_realtime.deepsort_tracker import DeepSort

############################
# (A) MobileFaceNet (InsightFace) 준비
############################
import insightface
from insightface.app import FaceAnalysis

app = FaceAnalysis(name="buffalo_s")  # "buffalo_m"은 MobileFaceNet 기반
app.prepare(ctx_id=0, det_size=(640,640))  # GPU라면 ctx_id=0, CPU는 -1

def get_mobileface_embedding(image_bgr):
    if image_bgr is None or image_bgr.size==0:
        return None
    faces = app.get(image_bgr)
    if len(faces)==0:
        return None
    return faces[0].embedding  # shape 예: (128,)

def is_my_face(face_embedding, my_embedding, threshold=0.4):
    sim = cosine_similarity([face_embedding], [my_embedding])[0][0]
    return (sim > threshold), sim

############################
# (B) 모델/함수 초기화
############################
warnings.filterwarnings("ignore", category=FutureWarning)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = YOLO("C:/Users/idea0/EE101/Jongsul/Yolomodels/epoch180.pt").to(device)
model_seg = YOLO("yolov8n-seg.pt").to(device)

# MobileFaceNet 임베딩
my_face_embedding = np.load("my_mobileface_embedding.npy")  # (128,) or (512,) etc.

############################
# (C) Mediapipe Pose 등 동일
############################
mp_pose = mp.solutions.pose
pose_danger = mp_pose.Pose(
    static_image_mode=True,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

mp_drawing = mp.solutions.drawing_utils
LEFT_SHOULDER = 11
RIGHT_SHOULDER= 12
LEFT_WRIST   = 15
RIGHT_WRIST  = 16

def is_arm_raised(shoulder_y, wrist_y, threshold=0.05):
    return wrist_y < (shoulder_y - threshold)

def boxes_overlap(boxA, boxB):
    (x1A, y1A, x2A, y2A) = boxA
    (x1B, y1B, x2B, y2B) = boxB
    overlap_x = not (x2A < x1B or x2B < x1A)
    overlap_y = not (y2A < y1B or y2B < y1A)
    return overlap_x and overlap_y

############################
# (D) DeepSORT
############################
tracker = DeepSort(
    max_age=30,
    n_init=3,
    nms_max_overlap=1.0,
    embedder='mobilenet',
    half=True,
    embedder_gpu=True
)

############################
# (E) 메인 루프
############################
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

dangerous_ids = set()
track_me_status = {}
track_arcface_count= {}
MAX_ARCFACE_FRAMES= 20
sim = 0

window_name = "DeepSORT + YOLO(SEG) + MobileFaceNet + Pose"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 960,720)

prev_time = time.time()
while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(rgb_frame, conf=0.5)
    results_seg = model_seg(rgb_frame, conf=0.5)

    # YOLO detection
    det= results[0]
    boxes2 = det.boxes
    masks2 = results_seg[0].masks

    person_detections= []
    weapon_boxes= []

    if boxes2 is not None:
        for i, box in enumerate(boxes2):
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls[0])
            conf = float(box.conf[0])

            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0),2)
            label = f"{model.names[class_id]}: {conf:.2f}"
            cv2.putText(frame, label,(x1,y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2)

            # 세그 윤곽선(사람만)
            if masks2 is not None and class_id==0:
                if i < len(masks2.data):
                    single_mask= masks2.data[i].cpu().numpy()
                    mask_bin= (single_mask>0.5).astype(np.uint8)
                    contours,_= cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    cv2.drawContours(frame, contours, -1, (0,255,255), 2)

            # DeepSORT
            if class_id==0:
                w= x2 - x1
                h= y2 - y1
                person_detections.append(((x1,y1,w,h), conf,0))
            elif class_id in [1,2]:
                weapon_boxes.append((x1,y1,x2,y2))

    # DeepSORT update
    tracks= tracker.update_tracks(person_detections, frame=rgb_frame)
    tracked_boxes=[]
    for t in tracks:
        if not t.is_confirmed() or t.time_since_update>1:
            continue
        tid= t.track_id
        l,t_,r,b_ = map(int,t.to_ltrb())
        tracked_boxes.append((tid,l,t_,r,b_))
        cv2.putText(frame,f"ID:{tid}", (l-10,t_-10),
                    cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,255),2)

    # MobileFaceNet 로직
    time1 = time.time()
    for (tid, px1, py1, px2, py2) in tracked_boxes:
        if tid not in track_me_status:
            track_me_status[tid]= False
        if tid not in track_arcface_count:
            track_arcface_count[tid]=0

        me_status= track_me_status[tid]
        #if me_status==False and track_arcface_count[tid]<MAX_ARCFACE_FRAMES:
        if True:
            print("Running MobileFacenet...")
            track_arcface_count[tid]+=1
            # MobileFaceNet 임베딩 추출
            PAD=10
            sub_face= frame[max(0,py1-PAD): py2+PAD, max(0,px1-PAD): px2+PAD]
            if sub_face.size==0:
                continue

            emb_image = get_mobileface_embedding(sub_face)
            if emb_image is not None:
                same_person, sim = is_my_face(emb_image, my_face_embedding, threshold=0.4)
                if same_person:
                    track_me_status[tid]=True
                    if tid in dangerous_ids:
                        dangerous_ids.remove(tid)

        # 시각화
        if track_me_status[tid]:
            text_arc= f"          Me(sim={sim:.2f})"
            color=(0,255,0)
        else:
            text_arc= "           NotMe(sim={sim:.2f})"
            color=(0,0,255)
        cv2.putText(frame,text_arc,(px1, py1-10),
                    cv2.FONT_HERSHEY_SIMPLEX,0.6,color,2)
    time2 = time.time()
    print("MobileFacenet running time: ", time2-time1)

    # 무기 교차 & NotMe => dangerous
    for (tid, px1, py1, px2, py2) in tracked_boxes:
        if not track_me_status[tid]:
            pbox= (px1, py1, px2, py2)
            for wb in weapon_boxes:
                if boxes_overlap(pbox, wb):
                    dangerous_ids.add(tid)
                    break

    # Dangerous => Mediapipe pose
    for (tid, px1, py1, px2, py2) in tracked_boxes:
        if tid in dangerous_ids:
            sub= frame[py1:py2, px1:px2]
            if sub.size==0:
                continue
            c_rgb= cv2.cvtColor(sub, cv2.COLOR_BGR2RGB)
            pose_result= pose_danger.process(c_rgb)
            if pose_result.pose_landmarks:
                lms= pose_result.pose_landmarks.landmark
                sub_w= px2 - px1
                sub_h= py2 - py1

                left_shoulder_y= lms[LEFT_SHOULDER].y
                right_shoulder_y= lms[RIGHT_SHOULDER].y
                left_wrist_y= lms[LEFT_WRIST].y
                right_wrist_y= lms[RIGHT_WRIST].y

                la_up= (left_wrist_y< (left_shoulder_y-0.05))
                ra_up= (right_wrist_y<(right_shoulder_y-0.05))
                if la_up and ra_up:
                    a_text= "both arms up"
                elif la_up:
                    a_text= "left arm up"
                elif ra_up:
                    a_text= "right arm up"
                else:
                    a_text= "do nothing"

                for lm in lms:
                    cx= px1+int(lm.x*sub_w)
                    cy= py1+int(lm.y*sub_h)
                    cv2.circle(frame,(cx,cy),3,(0,255,255),-1)
                cv2.putText(frame,f"Dangerous person: {a_text}",
                            (px1,py1+20),
                            cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)

    # FPS 표시
    now= time.time()
    fps= 1.0/(now - prev_time)
    prev_time= now
    cv2.putText(frame,f"FPS:{fps:.2f}",(10,30),
                cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

    cv2.imshow(window_name, frame)
    if cv2.waitKey(1)&0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\idea0/.insightface\models\buffalo_s\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127

Running MobileFacenet...
MobileFacenet running time:  0.14100146293640137

0: 480x640 1 personnnn, 103.0ms
Speed: 20.0ms preprocess, 103.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 121.0ms
Speed: 2.0ms preprocess, 121.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
Running MobileFacenet...
MobileFacenet running time:  0.10100889205932617

0: 480x640 1 personnnn, 95.0ms
Speed: 8.0ms preprocess, 95.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 124.0ms
Speed: 2.0ms preprocess, 124.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
Running MobileFacenet...
MobileFacenet running time:  0.13571715354919434

0: 480x640 1 personnnn, 84.0ms
Speed: 2.0ms preprocess, 84.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 136.0ms
Speed: 2.0ms preprocess, 136.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
Runn

In [1]:
import torch
import sys

print("python version:", sys.version)
print("torch version:", torch.__version__)
print("CUDA Version:", torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())

python version: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
torch version: 1.12.1+cu116
CUDA Version: 11.6
CUDA Available: False


In [18]:
#pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
print("사용 가능한 GPU 수:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))

사용 가능한 GPU 수: 0
